In [1]:
setwd("/projects/CARDIPS/analysis/epigenome_resource")
suppressPackageStartupMessages(source("analyses/jennifer/notebooks/functions.R"))

set.seed(5366)

In [2]:
tissues = c("iPSC", "PPC", "CVPC")
analyses = c("eqtls", "caqtls", "haqtls")

# Summarize and process GWAS colocalization results

## load manifest

In [3]:
manifest_file = "/projects/CARDIPS/analysis/epigenome_resource/analyses/jennifer/gwas_coloc/input/manifest.txt"
manifest = fread(manifest_file, data.table = F)

manifest %>% head(2)

,trait_id,trait_type,coding_description,description,full_trait_id,filename,exists,taskid
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>
1,healthspan_summary,continuous,,Healthspan,healthspan_summary,/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_liftover/hg38_summary_statistics/healthspan_summary.hg38.tsv.gz,TRUE,1
2,lifegen_phase2_bothpl_alldr_2017_09_18,continuous,,Longevity,lifegen_phase2_bothpl_alldr_2017_09_18,/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_liftover/hg38_summary_statistics/lifegen_phase2_bothpl_alldr_2017_09_18.hg38.tsv.gz,TRUE,2


## check if there's input

In [12]:
# for (t in tissues)
# {
#     for (a in analyses)
#     {
#         files = list.files(paste("analyses/jennifer/gwas_coloc", a, t, sep = "/"))
#         message(paste(t, a, length(files)), appendLF = F)
#     }
# }

## for each tissue and QTL type, summarize GWAS results 

In [ ]:
pipeline = "/projects/CARDIPS/analysis/epigenome_resource/analyses/jennifer/notebooks/08.04.summarize_gwas.R"

for (t in tissues)
{
    for (a in analyses)
    {
        files = list.files(paste("analyses/jennifer/gwas_coloc", a, t, sep = "/"))
        if (length(files) > 0)
        {
            cmd = paste("Rscript", pipeline, "--analysis", a, "--tissue", t)
            log_out = paste(getwd(), "analyses/jennifer/gwas_coloc/logs", paste(a, t, "summarize.out", sep = "_"), sep = "/")
            log_err = paste(getwd(), "analyses/jennifer/gwas_coloc/logs", paste(a, t, "summarize.err", sep = "_"), sep = "/")
            run_qsub(name = paste(a, t, sep = "_"), cmd = cmd, threads = 4, log_out = log_out, log_err = log_err, exec = T)
        }
    }
}


## aggregate each summary

In [18]:
summary = as.data.frame(rbindlist(lapply(tissues, function(t)
{
    as.data.frame(rbindlist(lapply(analyses, function(a)
    {
        message(paste(t, a), appendLF = F)
        file = paste("analyses/jennifer/gwas_coloc", paste(paste(a, t, "summary", sep = "_"), "txt", sep = "."), sep = "/")
        if (file.exists(file))
        {
            fread(file, data.table = F)
        }
    })))
}))) %>% mutate(p.gwas = as.double(p.gwas), p.eqtl = as.double(p.eqtl)) %>% dplyr::rename(type = discovery_order) %>% filter(type == 0)


iPSC eqtls
iPSC caqtls
iPSC haqtls
PPC eqtls
PPC caqtls
PPC haqtls
CVPC eqtls
CVPC caqtls
CVPC haqtls


In [19]:
table(summary$type)


      0 
1846395 

In [20]:
head(summary,2)

,analysis,tissue,qtl_id,element_id,type,trait_id,nsnps,PP.H0.abf,PP.H1.abf,PP.H2.abf,⋯,topsnp,topsnp_pp,beta.eqtl,se.eqtl,p.eqtl,beta.gwas,se.gwas,p.gwas,bonferroni.eqtl,cs_size
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,eqtls,iPSC,0-ENSG00000000460.17,ENSG00000000460.17,0,biomarkers-30690-both_sexes-irnt.eur,5301,0.1831085,0.6395308,0.02392711,⋯,VAR_1_169804722_A_C,0.01158138,0.8196584,0.1416713,1.546628e-09,-0.01193,0.004174,0.004255984,8.231152e-06,526
2,eqtls,iPSC,0-ENSG00000000460.17,ENSG00000000460.17,0,biomarkers-30690-both_sexes-irnt.meta,5301,0.1606204,0.5609879,0.03642126,⋯,VAR_1_169804722_A_C,0.01136916,0.8196584,0.1416713,1.546628e-09,-0.01259,0.004102,0.002152782,8.231152e-06,482


## add trait description

In [21]:
summary2 = merge(summary %>% dplyr::rename(full_trait_id = trait_id), 
                 manifest[,c("trait_id", "full_trait_id", "description")], by = "full_trait_id", all.x = T)

In [22]:
sub_manifest = fread("analyses/jennifer/gwas_independent/subset_manifest.txt", data.table = F)
sub_manifest %>% filter(!full_trait_id %in% summary2$full_trait_id)

trait_id,trait_type,coding_description,description,full_trait_id,filename,exists
<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<lgl>


In [23]:
unique(summary2$description)

[1] "alzeimer dimentia"                                                                    
  [2] "all psychiatric disorders"                                                            
  [3] "all psychiatric disorders (meta)"                                                     
  [4] "Cholesterol"                                                                          
  [5] "Glucose"                                                                              
  [6] "HDL cholesterol"                                                                      
  [7] "LDL direct"                                                                           
  [8] "birth weight (eur)"                                                                   
  [9] "birth weight (transethnic)"                                                           
 [10] "Cancer code, self-reported"                                                           
 [11] "Non-cancer illness code, self-reported"                                               
 [12] "simvastatin"                                                                          
 [13] "insulin product"                                                                      
 [14] "metformin"                                                                            
 [15] "Diabetes"                                                                             
 [16] "childhood obesity (eur)"                                                              
 [17] "Ventricular rate"                                                                     
 [18] "P duration"                                                                           
 [19] "QRS duration"                                                                         
 [20] "Birth weight"                                                                         
 [21] "Body mass index (BMI)"                                                                
 [22] "Weight"                                                                               
 [23] "White blood cell (leukocyte) count"                                                   
 [24] "Red blood cell (erythrocyte) count"                                                   
 [25] "Haemoglobin concentration"                                                            
 [26] "Haematocrit percentage"                                                               
 [27] "Mean corpuscular volume"                                                              
 [28] "Mean corpuscular haemoglobin"                                                         
 [29] "Mean corpuscular haemoglobin concentration"                                           
 [30] "Red blood cell (erythrocyte) distribution width"                                      
 [31] "Platelet count"                                                                       
 [32] "Platelet crit"                                                                        
 [33] "Mean platelet (thrombocyte) volume"                                                   
 [34] "Platelet distribution width"                                                          
 [35] "Lymphocyte count"                                                                     
 [36] "Monocyte count"                                                                       
 [37] "Neutrophill count"                                                                    
 [38] "Eosinophill count"                                                                    
 [39] "Basophill count"                                                                      
 [40] "Lymphocyte percentage"                                                                
 [41] "Monocyte percentage"                                                                  
 [42] "Neutrophill percentage"                                                               
 [43] "Eosinophill percentage"                        

## add gene and peak coordinates

In [24]:
element_info = fread("analyses/jennifer/summary_files/all.phenotype_info.txt", data.table = F)

In [25]:
head(element_info,2)

,element_chr,element_start,element_end,element_id,element_name,element_strand
,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,chr1,11869,14409,ENSG00000290825.1,DDX11L2,+
2,chr1,12010,13670,ENSG00000223972.6,DDX11L1,+


In [44]:
summary3 = merge(element_info, summary2, by = "element_id", all.y = T)
summary3$element_cond = paste(summary3$element_id, summary3$type, sep = "_")

# check that all elements have coordinates
summary3 %>% filter(is.na(element_start))

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


element_id,element_chr,element_start,element_end,element_name,element_strand,full_trait_id,analysis,tissue,qtl_id,⋯,se.eqtl,p.eqtl,beta.gwas,se.gwas,p.gwas,bonferroni.eqtl,cs_size,trait_id,description,element_cond
<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>


## filter for significant QTLs

In [45]:
all_qtls = fread("analyses/jennifer/summary_files/all.qtls.no_mhc.txt", data.table = F) %>% distinct() %>% filter(new_egene == T & type == 0)
head(all_qtls,3)

length(unique(all_qtls$element_cond))

,element_id,element_chr,element_start,element_end,element_name,element_strand,type,id,beta,se,⋯,fdr,qval,filt_qval,egene,new_egene,tissue,analysis,element_cond,pos,distance_from_tss
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,cvpc_atac_peak_10001,chr1,76511208,76511727,cvpc_atac_peak_10001,,0,VAR_1_76511517_G_A,-1.3113800,0.2110911,⋯,5.070823e-09,5.525888e-07,5.543115e-07,TRUE,TRUE,CVPC,caqtls,cvpc_atac_peak_10001_0,76511517,NA
2,cvpc_atac_peak_100016,chr16,83066964,83067348,cvpc_atac_peak_100016,,0,VAR_16_83050845_T_A,-1.5819262,0.3322844,⋯,3.768682e-04,1.089547e-02,1.090663e-02,TRUE,TRUE,CVPC,caqtls,cvpc_atac_peak_100016_0,83050845,NA
3,cvpc_atac_peak_100021,chr16,83146307,83146659,cvpc_atac_peak_100021,,0,VAR_16_83143805_T_C,0.9875124,0.1916778,⋯,3.345122e-05,1.416101e-03,1.418007e-03,TRUE,TRUE,CVPC,caqtls,cvpc_atac_peak_100021_0,83143805,NA


[1] 60510

In [46]:
# check that all qtls are present
table(all_qtls$tissue, all_qtls$analysis)

      
       caqtls eqtls haqtls
  CVPC  11239  4837   8937
  iPSC   9053  9012   1663
  PPC   10313  5456      0

In [47]:
summary4 = summary3 

summary4$element_cond = ifelse(summary4$analysis == "eqtls", 
                               paste(tolower(summary4$tissue), summary4$element_cond, sep = "_"), 
                               summary4$element_cond)

# summary4 = summary4 %>% filter(element_cond %in% all_qtls$element_cond)

In [49]:
message(paste("Did not test:", length(unique(all_qtls[!all_qtls$element_cond %in% summary4$element_cond,]$element_cond))))

# These could not be tested because overlap with gwas  < 50 snps or no GWAS peaks below significance
head(all_qtls[!all_qtls$element_cond %in% summary4$element_cond,]$element_cond)

Did not test: 279



[1] "cvpc_atac_peak_103546_0" "cvpc_atac_peak_103548_0"
[3] "cvpc_atac_peak_103779_0" "cvpc_atac_peak_132767_0"
[5] "cvpc_atac_peak_132889_0" "cvpc_atac_peak_132901_0"

## add TFBS info

In [50]:
# hocomoco_tfbs = fread("analyses/tim/tobias/all_tfbs.txt", data.table = F)
# summary4$in_hocomoco_tfbs = ifelse(summary4$element_id %in% hocomoco_tfbs$V10, T, F)
# table(summary4$in_hocomoco_tfbs)

In [51]:
# jaspar_tfbs = fread("analyses/tim/tobias/all_jaspar_tfbs.txt", data.table = F)
# summary4$in_jaspar_tfbs = ifelse(summary4$element_id %in% jaspar_tfbs$V10, T, F)
# table(summary4$in_jaspar_tfbs)

## final TFBS (JASPAR and Hocomoco Nanog)

In [52]:
# final_tfbs = rbind(jaspar_tfbs %>% dplyr::rename(Model = V4), hocomoco_tfbs %>% filter(TF %like% "NANOG") %>% select(-TF))
# head(final_tfbs,2)
# summary4$in_final_tfbs = ifelse(summary4$element_id %in% final_tfbs$V10, T, F)

## add cluster info

In [53]:
mods = fread("analyses/jennifer/summary_files/all.qtl_modules.H4_0.8.txt", data.table = F)
mods$element_cond = ifelse(mods$element_cond %like% "ENSG", 
                           paste(tolower(mods$tissue), mods$element_cond, sep = "_"), 
                           mods$element_cond)
head(mods,2)

,cluster_id,element_id,element_cond,degree,n_members,n_occur,tissue,qtl_type,qtl_combo,module,n_qtltypes
,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>
1,CVPC_1,cvpc_atac_peak_22691,cvpc_atac_peak_22691_0,3,10,1,CVPC,ATAC,ATAC-ChIP-RNA,module,3
2,CVPC_1,cvpc_atac_peak_22694,cvpc_atac_peak_22694_0,5,10,1,CVPC,ATAC,ATAC-ChIP-RNA,module,3


In [61]:
summary5 = merge(summary4, 
                mods %>% select(element_cond, cluster_id, degree, n_members, n_occur, tissue, qtl_type, qtl_combo, module, n_qtltypes), 
                by = c("element_cond", "tissue"), all.x = T)

In [62]:
# remove conditional qtls
summary5 %>% filter(is.na(cluster_id))

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


element_cond,tissue,element_id,element_chr,element_start,element_end,element_name,element_strand,full_trait_id,analysis,⋯,trait_id,description,cluster_id,degree,n_members,n_occur,qtl_type,qtl_combo,module,n_qtltypes
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>


In [63]:
str(summary5)

'data.frame':	1846395 obs. of  41 variables:
 $ element_cond   : chr  "cvpc_atac_peak_10001_0" "cvpc_atac_peak_10001_0" "cvpc_atac_peak_10001_0" "cvpc_atac_peak_10001_0" ...
 $ tissue         : chr  "CVPC" "CVPC" "CVPC" "CVPC" ...
 $ element_id     : chr  "cvpc_atac_peak_10001" "cvpc_atac_peak_10001" "cvpc_atac_peak_10001" "cvpc_atac_peak_10001" ...
 $ element_chr    : chr  "chr1" "chr1" "chr1" "chr1" ...
 $ element_start  : int  76511208 76511208 76511208 76511208 76511208 76511208 76511208 76511208 76511208 76511208 ...
 $ element_end    : int  76511727 76511727 76511727 76511727 76511727 76511727 76511727 76511727 76511727 76511727 ...
 $ element_name   : chr  "cvpc_atac_peak_10001" "cvpc_atac_peak_10001" "cvpc_atac_peak_10001" "cvpc_atac_peak_10001" ...
 $ element_strand : chr  "" "" "" "" ...
 $ full_trait_id  : chr  "icd10-I44-both_sexes.eur" "icd10-I95-both_sexes.eur" "continuous-30300-both_sexes-irnt.meta" "biomarkers-30760-both_sexes-irnt.eur" ...
 $ analysis       : chr  "caq

## for each cluster and trait, what is the proportion that colocalized?

In [64]:
tmp = summary5 %>% 
    filter(topsnp_pp >= 0.01 & 
           PP.H4.abf >= 0.8 & 
           p.eqtl <= 5e-05 & 
           p.gwas <= 5e-08) %>% 
    select(full_trait_id, element_cond, cluster_id) %>% distinct()

head(tmp,2)

tmp = data.frame(table(tmp$full_trait_id, tmp$cluster_id))
colnames(tmp) = c("full_trait_id", "cluster_id", "number_elements_coloc")
head(tmp,2)

summary6 = merge(summary5, tmp, by = c("full_trait_id", "cluster_id"), all.x = T) 

# for clusters that had zero elements colocalized, set to 0
summary6[is.na(summary6$number_elements_coloc),]$number_elements_coloc = 0

summary6$prop_cluster_coloc = summary6$number_elements_coloc / summary6$n_members * 100

summary6 %>% filter(is.na(number_elements_coloc))
summary6 %>% filter(is.na(prop_cluster_coloc))


,full_trait_id,element_cond,cluster_id
,<chr>,<chr>,<chr>
1,continuous-30180-both_sexes-irnt.meta,cvpc_atac_peak_100536_0,CVPC_172
2,continuous-30110-both_sexes-irnt.meta,cvpc_atac_peak_100536_0,CVPC_172


,full_trait_id,cluster_id,number_elements_coloc
,<fct>,<fct>,<int>
1,AD_sumstats_Jansenetal_2019sept,CVPC_10007,0
2,all_psychiatric_disorders.meta,CVPC_10007,0


Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


full_trait_id,cluster_id,element_cond,tissue,element_id,element_chr,element_start,element_end,element_name,element_strand,⋯,description,degree,n_members,n_occur,qtl_type,qtl_combo,module,n_qtltypes,number_elements_coloc,prop_cluster_coloc
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>


Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


full_trait_id,cluster_id,element_cond,tissue,element_id,element_chr,element_start,element_end,element_name,element_strand,⋯,description,degree,n_members,n_occur,qtl_type,qtl_combo,module,n_qtltypes,number_elements_coloc,prop_cluster_coloc
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>


In [65]:
summary(summary6$number_elements_coloc)
summary(summary6$prop_cluster_coloc)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00000 0.00000 0.00000 0.01447 0.00000 9.00000 

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
  0.0000   0.0000   0.0000   0.7343   0.0000 100.0000 

## annotate which cluster colocalized?

In [66]:
tmp = summary6
tmp$coloc_gwas = ifelse(tmp$topsnp_pp >= 0.01 & 
                        tmp$PP.H4.abf >= 0.8 & 
                        tmp$p.eqtl <= 5e-5 & 
                        tmp$p.gwas <= 5e-8 , 
#                         tmp$prop_cluster_coloc >= 50, 
                        T, F) 

tmp$cluster_gwas = paste(tmp$cluster_id, tmp$full_trait_id)

summary6 = summary6 %>% 
    mutate(cluster_gwas = paste(cluster_id, full_trait_id)) %>%
    mutate(coloc_gwas = ifelse(cluster_gwas %in% tmp[tmp$coloc_gwas == T,]$cluster_gwas, T, F)) %>%
    select(-cluster_gwas)

## annotate which QTLs are fetal-unique. for fetal-unique QTLs that were in the same module as an adult-shared QTL, annotate them as adult-shared.

In [67]:
ld = fread("analyses/jennifer/summary_files/fetal_unique.txt", data.table = F)
head(ld,2)

,element_cond,cluster_id,tissue,analysis,fetal_unique,fetal_unique_mod,element_id,type
,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<int>
1,cvpc_atac_peak_10001_0,CVPC_2928,CVPC,caqtls,FALSE,FALSE,cvpc_atac_peak_10001,0
2,cvpc_atac_peak_10001_1,,CVPC,caqtls,TRUE,TRUE,cvpc_atac_peak_10001,1


In [68]:
summary6$fetal_unique = ifelse(summary6$cluster_id %in% ld[ld$fetal_unique == F,]$cluster_id, F, T)

table(summary6$fetal_unique)


  FALSE    TRUE 
1654058  192337 

In [69]:
a = summary6 %>% filter(fetal_unique == T & coloc_gwas == T)
message(paste("How many fetal-unique QTLs that colocalize with GWAS:", length(unique(a$element_cond))))

How many fetal-unique QTLs that colocalize with GWAS: 102



In [70]:
# summary6 %>% filter(fetal_unique == T & coloc_gwas == T) %>% 
#     select(cluster_id, full_trait_id, description, coloc_gwas, fetal_unique) %>% 
#     distinct() %>% filter(full_trait_id %in% sub_manifest$full_trait_id) %>%
#     arrange(description) %>% arrange(cluster_id)

## save all results

In [71]:
message("Saving..", appendLF = F)
file = "analyses/jennifer/gwas_coloc/all.gwas_summary.2024_0321.txt"
fwrite(summary6, file, row.names = F, sep = "\t")
message(paste("Saved:", file), appendLF = F)

Saving..
Saved: analyses/jennifer/gwas_coloc/all.gwas_summary.2024_0321.txt


## annotate which capeaks have a TFBS, including hapeaks that overlap with a TFBS capeak

In [72]:
tfbs = fread("analyses/jennifer/summary_files/final_tfbs.expressed.txt", data.table = F)

In [73]:
tfbs2 = as.data.frame(rbindlist(lapply(c("ppc", "cvpc", "ipsc"), function(tiss)
{
    this = tfbs %>% filter(V10 %like% tiss)
    this = this[this[,paste0(tiss, "_exp")] == T,]
    return(this)
})))

In [74]:
head(tfbs2,2)

,Model,V10,TF,cvpc_exp,ipsc_exp,ppc_exp,motif_id
,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<chr>
1,AhrArnt_MA0006.1,ppc_atac_peak_4,AHR,TRUE,TRUE,TRUE,MA0006.1
2,AhrArnt_MA0006.1,ppc_atac_peak_4,ARNT,TRUE,TRUE,TRUE,MA0006.1


In [75]:
int = fread("analyses/jennifer/summary_files/cvpc_ipsc.atac_chip.intersect.bed", data.table = F)
int = int[int$V4 %in% tfbs2$V10,]
head(int,2)

,V1,V2,V3,V4,V5,V6,V7,V8
,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>
1,chr1,826692,827857,cvpc_atac_peak_2,chr1,825773,827873,cvpc_chip_peak_2
4,chr1,959054,959586,cvpc_atac_peak_37,chr1,958564,959276,cvpc_chip_peak_5


In [76]:
summary6$has_tfbs = ifelse(summary6$element_cond %in% tfbs2$V10 | summary6$element_cond %in% int$V8, T, F)

# filter for downstream traits

In [77]:
subset_manifest = fread("analyses/jennifer/gwas_independent/subset_manifest.txt", data.table = F) %>%
    filter(trait_id != "healthspan_summary" & full_trait_id != "continuous-20022-both_sexes-irnt.meta")

summary7 = summary6 %>% filter(full_trait_id %in% subset_manifest$full_trait_id)

file = "analyses/jennifer/gwas_coloc/all.gwas_summary.all_downstream_traits.2024_0321.txt"
fwrite(summary7, file, row.names = F, sep = "\t")
message(paste("Saved:", file), appendLF = F)

Saved: analyses/jennifer/gwas_coloc/all.gwas_summary.all_downstream_traits.2024_0321.txt


In [78]:
# how many qtls are fetal-unique and colocalized with GWAS
a = summary7 %>% filter(fetal_unique == T & coloc_gwas == T)
length(unique(a$element_cond))

[1] 14

In [79]:
table(a %>% select(element_cond, module) %>% distinct() %>% pull(module))


   module singleton 
        2        12 